# Setup Tools for Sales Assistant Agent
We are going to setup all the tools that will be used by the Agent

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Tools for Unstructured Data (Cortex Search)
We are going to setup two tools tha will help the agent to extract information from unstructured data. One will be using documents for bikes while the other will be using documents for ski.The information is stored in PDF format so first we are going to process it.

### Bikes Documentation

We are going to process the documents about bikes and create a Cortex Search service that will serve as a Tool for the Agent in order to retrieve information bout bikes when needed

In [ ]:
-- Check what documents are available

SELECT * FROM DIRECTORY('@DOCS_BIKES');

Read the PDF File

In [ ]:
CREATE OR REPLACE TABLE RAW_TEXT_BIKES AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@DOCS_BIKES',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@DOCS_BIKES');

SELECT * FROM RAW_TEXT_BIKES;

Split the PDF file into pieces

In [ ]:
-- Create chunks from extracted content

CREATE OR REPLACE TABLE CHUNKED_TEXT_BIKES AS
SELECT
   RELATIVE_PATH,
   c.INDEX::INTEGER AS CHUNK_INDEX,
   c.value::TEXT AS CHUNK_TEXT
FROM
   RAW_TEXT_BIKES,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      EXTRACTED_LAYOUT,
      'markdown',
      1512,
      256,
      ['\n\n', '\n', ' ', '']
   )) c;

SELECT * FROM CHUNKED_TEXT_BIKES;

#### Create Cortex Search Service on Bikes Docs

Create the Cortex Search Service that will be used as Tool for the agent.

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE BIKES_RAG_TOOL
  ON CHUNK_TEXT
  ATTRIBUTES RELATIVE_PATH, CHUNK_INDEX
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      CHUNK_TEXT,
      RELATIVE_PATH,
      CHUNK_INDEX
  FROM CHUNKED_TEXT_BIKES
);

### Ski Documentation

Let's create another Tool to retreive information about Ski Documents

In [ ]:
-- Check what documents are available

SELECT * FROM DIRECTORY('@DOCS_SKI');

In [ ]:
CREATE OR REPLACE TABLE RAW_TEXT_SKI AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@DOCS_SKI',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@DOCS_SKI');

SELECT * FROM RAW_TEXT_SKI;

In [ ]:
-- Create chunks from extracted content

CREATE OR REPLACE TABLE CHUNKED_TEXT_SKI AS
SELECT
   RELATIVE_PATH,
   c.INDEX::INTEGER AS CHUNK_INDEX,
   c.value::TEXT AS CHUNK_TEXT
FROM
   RAW_TEXT_SKI,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      EXTRACTED_LAYOUT,
      'markdown',
      1512,
      256,
      ['\n\n', '\n', ' ', '']
   )) c;

SELECT * FROM CHUNKED_TEXT_SKI;

#### Create Cortex Search Service on Ski Docs

Create the Cortex Search Service that will be used as Tool for the agent.

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE SKI_RAG_TOOL
  ON CHUNK_TEXT
  ATTRIBUTES RELATIVE_PATH, CHUNK_INDEX
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      CHUNK_TEXT,
      RELATIVE_PATH,
      CHUNK_INDEX
  FROM CHUNKED_TEXT_SKI
);

#### Test the API for the Tool
You can test the service is working as expected before configuring the Cortex Agent

In [ ]:
from snowflake.core import Root
from snowflake.cortex import complete

root = Root(get_active_session())

question = 'Who tested the downhill bike'

# Fetch service
my_service = (root
  .databases["CC_CORTEX_AGENTS_SUMMIT"]
  .schemas["PUBLIC"]
  .cortex_search_services["BIKES_RAG_TOOL"]
)

# Query service
resp = my_service.search(
  query=question,
  columns=["CHUNK_INDEX", "CHUNK_TEXT", "RELATIVE_PATH"],
  limit=1
)
resp = resp.results[0]

st.info(f'**File:** {resp["RELATIVE_PATH"]}\n\n {resp["CHUNK_TEXT"]}')

# Generate Response
model = 'mistral-large2'
prompt = f"{question} Answer based on the provided context: {resp['CHUNK_TEXT']}"
response = complete(model, prompt).strip()

st.info(f'**LLM Response:**\n\n**{response}**')

## Tools for Structured Data (Cortex Analyst)

Another Tool that we will be providing to the Cortex Agent will be Cortex Analyst which will provide the capability to extract information from Snowflake Tables.  In the API call we will be providing the location of a Semantic file that contains information about the business terminology to describe the data.

First we are going to create some syntethic data about the bikes and ski products that we have.

We are going to create the following tables with content:


**DIM_ARTICLE – Article/Item Dimension**

Purpose: Stores descriptive information about the products (articles) being sold.

Key Columns:

- ARTICLE_ID (Primary Key): Unique identifier for each article.
- ARTICLE_NAME: Full name/description of the product.
- ARTICLE_CATEGORY: Product category (e.g., Bike, Skis, Ski Boots).
- ARTICLE_BRAND: Manufacturer or brand (e.g., Mondracer, Carver).
- ARTICLE_COLOR: Dominant color for the article.
- ARTICLE_PRICE: Standard unit price of the article.


**DIM_CUSTOMER – Customer Dimension**

Purpose: Contains demographic and segmentation info about each customer.

Key Columns:

- CUSTOMER_ID (Primary Key): Unique identifier for each customer.
- CUSTOMER_NAME: Display name for the customer.
- CUSTOMER_REGION: Geographic region (e.g., North, South).
- CUSTOMER_AGE: Age of the customer.
- CUSTOMER_GENDER: Gender (Male/Female).
- CUSTOMER_SEGMENT: Marketing segment (e.g., Premium, Regular, Occasional).


**FACT_SALES – Sales Transactions Fact Table**

Purpose: Captures individual sales transactions (facts) with references to article and customer details.

Key Columns:

- SALE_ID (Primary Key): Unique identifier for the transaction.
- ARTICLE_ID (Foreign Key): Links to DIM_ARTICLE.
- CUSTOMER_ID (Foreign Key): Links to DIM_CUSTOMER.
- DATE_SALES: Date when the sale occurred.
- QUANTITY_SOLD: Number of units sold in the transaction.
- TOTAL_PRICE: Total transaction value (unit price × quantity).
- SALES_CHANNEL: Sales channel used (e.g., Online, In-Store, Partner).
- PROMOTION_APPLIED: Boolean indicating if the sale involved a promotion or discount.



In [ ]:
CREATE OR REPLACE TABLE DIM_ARTICLE (
    ARTICLE_ID INT PRIMARY KEY,
    ARTICLE_NAME STRING,
    ARTICLE_CATEGORY STRING,
    ARTICLE_BRAND STRING,
    ARTICLE_COLOR STRING,
    ARTICLE_PRICE FLOAT
);

INSERT INTO DIM_ARTICLE (ARTICLE_ID, ARTICLE_NAME, ARTICLE_CATEGORY, ARTICLE_BRAND, ARTICLE_COLOR, ARTICLE_PRICE)
VALUES 
(1, 'Mondracer Infant Bike', 'Bike', 'Mondracer', 'Red', 3000),
(2, 'Premium Bicycle', 'Bike', 'Veloci', 'Blue', 9000),
(3, 'Ski Boots TDBootz Special', 'Ski Boots', 'TDBootz', 'Black', 600),
(4, 'The Ultimate Downhill Bike', 'Bike', 'Graviton', 'Green', 10000),
(5, 'The Xtreme Road Bike 105 SL', 'Bike', 'Xtreme', 'White', 8500),
(6, 'Carver Skis', 'Skis', 'Carver', 'Orange', 790),
(7, 'Outpiste Skis', 'Skis', 'Outpiste', 'Yellow', 900),
(8, 'Racing Fast Skis', 'Skis', 'RacerX', 'Blue', 950);

In [ ]:
CREATE OR REPLACE TABLE DIM_CUSTOMER (
    CUSTOMER_ID INT PRIMARY KEY,
    CUSTOMER_NAME STRING,
    CUSTOMER_REGION STRING,
    CUSTOMER_AGE INT,
    CUSTOMER_GENDER STRING,
    CUSTOMER_SEGMENT STRING
);

INSERT INTO DIM_CUSTOMER (CUSTOMER_ID, CUSTOMER_NAME, CUSTOMER_REGION, CUSTOMER_AGE, CUSTOMER_GENDER, CUSTOMER_SEGMENT)
SELECT 
    SEQ4() AS CUSTOMER_ID,
    'Customer ' || SEQ4() AS CUSTOMER_NAME,
    CASE MOD(SEQ4(), 5)
        WHEN 0 THEN 'North'
        WHEN 1 THEN 'South'
        WHEN 2 THEN 'East'
        WHEN 3 THEN 'West'
        ELSE 'Central'
    END AS CUSTOMER_REGION,
    UNIFORM(18, 65, RANDOM()) AS CUSTOMER_AGE,
    CASE MOD(SEQ4(), 2)
        WHEN 0 THEN 'Male'
        ELSE 'Female'
    END AS CUSTOMER_GENDER,
    CASE MOD(SEQ4(), 3)
        WHEN 0 THEN 'Premium'
        WHEN 1 THEN 'Regular'
        ELSE 'Occasional'
    END AS CUSTOMER_SEGMENT
FROM TABLE(GENERATOR(ROWCOUNT => 5000));

In [ ]:
CREATE OR REPLACE TABLE FACT_SALES (
    SALE_ID INT PRIMARY KEY,
    ARTICLE_ID INT,
    DATE_SALES DATE,
    CUSTOMER_ID INT,
    QUANTITY_SOLD INT,
    TOTAL_PRICE FLOAT,
    SALES_CHANNEL STRING,
    PROMOTION_APPLIED BOOLEAN,
    FOREIGN KEY (ARTICLE_ID) REFERENCES DIM_ARTICLE(ARTICLE_ID),
    FOREIGN KEY (CUSTOMER_ID) REFERENCES DIM_CUSTOMER(CUSTOMER_ID)
);

-- Populating Sales Fact Table with new attributes
INSERT INTO FACT_SALES (SALE_ID, ARTICLE_ID, DATE_SALES, CUSTOMER_ID, QUANTITY_SOLD, TOTAL_PRICE, SALES_CHANNEL, PROMOTION_APPLIED)
SELECT 
    SEQ4() AS SALE_ID,
    A.ARTICLE_ID,
    DATEADD(DAY, UNIFORM(-1095, 0, RANDOM()), CURRENT_DATE) AS DATE_SALES,
    UNIFORM(1, 5000, RANDOM()) AS CUSTOMER_ID,
    UNIFORM(1, 10, RANDOM()) AS QUANTITY_SOLD,
    UNIFORM(1, 10, RANDOM()) * A.ARTICLE_PRICE AS TOTAL_PRICE,
    CASE MOD(SEQ4(), 3)
        WHEN 0 THEN 'Online'
        WHEN 1 THEN 'In-Store'
        ELSE 'Partner'
    END AS SALES_CHANNEL,
    CASE MOD(SEQ4(), 4)
        WHEN 0 THEN TRUE
        ELSE FALSE
    END AS PROMOTION_APPLIED
FROM DIM_ARTICLE A
JOIN TABLE(GENERATOR(ROWCOUNT => 10000)) ON TRUE
ORDER BY DATE_SALES;

### Semantic File

We are providing an already created Semantic File for your usage, but please explore it using the Semantic Model Generator. You have an explanation in the mean README of this Hands-On Lab

In [ ]:
create or replace stage semantic_files ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

COPY FILES
    INTO @semantic_files/
    FROM @CC_CORTEX_AGENTS_SUMMIT.PUBLIC.git_repo/branches/main/
    PATTERN='semantic.yaml';

## Improving Tool Usage with Dynamic Literal Retrieval

Thank you to the Cortex Analyst integration with Cortex Search, we can improve the retrieval of all possible values of a column. Instead of listing all the possible values in the semantic file, we can use Cortex Search Integration. 

Let's use it as example for the ARTICLE NAMES, so we do not have to list all.



In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH
  ON ARTICLE_NAME
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT ARTICLE_NAME
  FROM DIM_ARTICLE
);